In [1]:
# Ensure your working directory is: https://github.com/henriquebecker91/phd/tree/master/latex/revised_PPG2KP
import Pkg
Pkg.activate(".")
Pkg.instantiate()
using WebIO
WebIO.install_jupyter_nbextension()
using DataFrames
using DataFramesMeta
using Gadfly
using IJulia
using Weave
using CSV
using Revise
using TableView
using Printf
using PrettyTables

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-1451584982111981205\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2307205004421446846\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-1451584982111981205\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

 Activating environment at `~/Aulas/doutorado/phd/latex/revised_PPG2KP/Project.toml`
┌ Info: Installing Jupyter WebIO extension...
│   cmd = `/usr/bin/jupyter nbextension install --user /home/henrique/.julia/packages/WebIO/nTMDV/deps/bundles/webio-jupyter-notebook.js`
└ @ WebIO /home/henrique/.julia/packages/WebIO/nTMDV/deps/jupyter.jl:237
Up to date: /home/henrique/.local/share/jupyter/nbextensions/webio-jupyter-notebook.js

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable <the entry point> --user
    
┌ Info: Enabling Jupyter WebIO extension...
│   cmd = `/usr/bin/jupyter nbextension enable --user webio-jupyter-notebook`
└ @ WebIO /home/henrique/.julia/packages/WebIO/nTMDV/deps/jupyter.jl:241
Enabling notebook extension webio-jupyter-notebook...
      - Validating: OK


In [2]:
# Read the data, show nothing.
lp_method_csv_path = "./data/lp_method.csv"
lp_method_df = DataFrame(CSV.File(lp_method_csv_path))
#showtable(lp_method_df)
nothing

In [3]:
# Clean the data a little. Try to keep this safe to re-apply to a cleaned dataframe, if possible.
lp_method_df = let
    # Keep only the instance name (not the path).
    @with(lp_method_df, :instance_name .= basename.(:instance_name))
    @with(lp_method_df, :datafile .= basename.(:datafile))
    # Replace parameter numeric values with more descriptive strings.
    lp_method_code2name = Dict{Int, String}(1 => "simplex", 2 => "barrier")
    lp_switch_code2name = Dict{Int, String}(-2 => "only", 1 => "switch")
    if eltype(lp_method_df.lp_method) == Int
        lp_method_df.lp_method = getindex.((lp_method_code2name,), lp_method_df.lp_method)
    end
    if eltype(lp_method_df.lp_method_switch) == Int
        lp_method_df.lp_method_switch = getindex.((lp_switch_code2name,), lp_method_df.lp_method_switch)
    end
    # Change the not-a-number values in final_root_time to zero. They happen  
    lp_method_df
end
nothing

In [4]:
# Shows the cleaned data.
println(names(lp_method_df))
showtable(lp_method_df)

[:instance_name, :lp_method, :lp_method_switch, :pricing_method, :restricted_root_time, :final_root_time, :build_and_solve_time, :total_instance_time, :final_pricing_time, :iterated_pricing_time, :restricted_pricing_time, :total_pricing_time, :finished, :datafile]


Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[""], Dict{Symbol,Any}(:style => Dict("height" => "0","padding" => "0","background-color" => "#F5F7F7","color" => "#1c1f20","font-family" => "-apple-system, BlinkMacSystemFont, \"Segoe UI\", Roboto, Oxygen-Sans, Ubuntu, Cantarell, \"Helvetica Neue\", sans-serif"))), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id => "grid-0f3a79de",:className => "ag-theme-balham",:style => Dict("height" => "500px","width" => "100%")))], Dict{Symbol,Any}()), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("onCellKeyDown" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onRowClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellDoubleClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellFocused" => (Observable{Any} with 0 listeners. Value:
nothing, nothing)), Set(String[]), nothing, Asset[Asset("js", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/src/rowNumberRenderer.js"), Asset("js", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid.js"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid.css"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid-light.css"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid-dark.css")], Dict{Any,Any}(), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007fd7e2ab1600, Task (runnable) @0x00007fd7e2ab1600), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = ((function (RowNumberRenderer,agGrid){var gridOptions={\"onCellFocused\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellFocused\",\"scope\":\"4350808324289968833\",\"id\":\"ob_04\",\"type\":\"observable\"},x)}),\"onCellValueChanged\":(function (){return }),\"onCellDoubleClicked\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellDoubleClicked\",\"scope\":\"4350808324289968833\",\"id\":\"ob_02\",\"type\":\"observable\"},x)}),\"multiSortKey\":\"ctrl\",\"columnDefs\":[{\"filter\":false,\"editable\":false,\"headerTooltip\":\"\",\"field\":\"__row__\",\"headerName\":\"Row\",\"sortable\":true,\"type\":\"numericColumn\",\"cellRenderer\":\"rowNumberRenderer\",\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"instance_name\",\"headerName\":\"instance_name\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"lp_method\",\"headerName\":\"lp_method\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"lp_method_switch\",\"headerName\":\"lp_method_switch\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"pricing_method\",\"headerName\":\"pricing_method\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":\"agNumberColumnFilter\",\"editable\":false,\"headerTooltip\":\"Float64\",\"field\":\"restricted_root_time\",\"headerName\":\"restricted_root_time\",\"sortable\":true,\"type\":\"numericColumn\",\"resizable\":true},{\"filter\":\"agNumberColumnFilter\",\"editable\":false,\"headerTooltip\":\"Float64\",\"field\":\"final_root_time\",\"headerN

In [196]:
# Just checking which instances ended in timeout.
@linq lp_method_df |>
    where(:finished .== false) |>
    select(:instance_name, :lp_method, :lp_method_switch, :pricing_method, :total_instance_time)

,instance_name,lp_method,lp_method_switch,pricing_method,total_instance_time
,String,String,String,String,Float64
1,STS4,barrier,only,furini,NaN
2,okp4,barrier,only,furini,NaN
3,A5,barrier,only,furini,NaN
4,okp5,barrier,only,furini,NaN
5,okp1,barrier,only,furini,NaN
6,STS4s,barrier,only,furini,NaN


In [197]:
sel_columns = [:instance_name, :lp_method, :lp_method_switch, :pricing_method, :build_and_solve_time]
# long (opposed to wide) table of total time
lttt = select(lp_method_df, sel_columns)
lttt.params = join.(zip(lttt.lp_method, lttt.lp_method_switch, lttt.pricing_method), "_")
pretty_params = Dict{String, String}(
    "barrier_only_none" => "NP B",
    "barrier_switch_furini" => "FP DS+B",
    "simplex_only_furini" => "FP DS",
    "barrier_only_furini" => "FP B",
    "simplex_only_none" => "NP DS"
)
lttt.params = getindex.((pretty_params,), lttt.params)
lttt = select(lttt, :instance_name, :params, :build_and_solve_time)
lttt.build_and_solve_time = (x -> @sprintf("%.2f", x)).(lttt.build_and_solve_time)
lttt.build_and_solve_time = ifelse.(
    lttt.build_and_solve_time .== "NaN", "--", lttt.build_and_solve_time
)
# Table with instance, params, and time, in long format.
showtable(lttt)

Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[""], Dict{Symbol,Any}(:style => Dict("height" => "0","padding" => "0","background-color" => "#F5F7F7","color" => "#1c1f20","font-family" => "-apple-system, BlinkMacSystemFont, \"Segoe UI\", Roboto, Oxygen-Sans, Ubuntu, Cantarell, \"Helvetica Neue\", sans-serif"))), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id => "grid-249d2710",:className => "ag-theme-balham",:style => Dict("height" => "500px","width" => "100%")))], Dict{Symbol,Any}()), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("onCellKeyDown" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onRowClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellDoubleClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellFocused" => (Observable{Any} with 0 listeners. Value:
nothing, nothing)), Set(String[]), nothing, Asset[Asset("js", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/src/rowNumberRenderer.js"), Asset("js", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid.js"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid.css"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid-light.css"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid-dark.css")], Dict{Any,Any}(), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f783f55ba90, Task (runnable) @0x00007f783f55ba90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = ((function (RowNumberRenderer,agGrid){var gridOptions={\"onCellFocused\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellFocused\",\"scope\":\"14327599217283468713\",\"id\":\"ob_159\",\"type\":\"observable\"},x)}),\"onCellValueChanged\":(function (){return }),\"onCellDoubleClicked\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellDoubleClicked\",\"scope\":\"14327599217283468713\",\"id\":\"ob_157\",\"type\":\"observable\"},x)}),\"multiSortKey\":\"ctrl\",\"columnDefs\":[{\"filter\":false,\"editable\":false,\"headerTooltip\":\"\",\"field\":\"__row__\",\"headerName\":\"Row\",\"sortable\":true,\"type\":\"numericColumn\",\"cellRenderer\":\"rowNumberRenderer\",\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"instance_name\",\"headerName\":\"instance_name\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"params\",\"headerName\":\"params\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"build_and_solve_time\",\"headerName\":\"build_and_solve_time\",\"sortable\":true,\"type\":null,\"resizable\":true}],\"onRowClicked\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onRowClicked\",\"scope\":\"14327599217283468713\",\"id\":\"ob_158\",\"type\":\"observable\"},x)}),\"onCellKeyDown\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return 

In [198]:
# Table with the build+solve time for each instance (rows) and parameter combination (columns).
# Note this counts the theoretically unnecessary root node solving of the priced models.
# wide (opposed to long) table of total time
wttt = unstack(lttt, :instance_name, :params, :build_and_solve_time)

showtable(wttt)


Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[""], Dict{Symbol,Any}(:style => Dict("height" => "0","padding" => "0","background-color" => "#F5F7F7","color" => "#1c1f20","font-family" => "-apple-system, BlinkMacSystemFont, \"Segoe UI\", Roboto, Oxygen-Sans, Ubuntu, Cantarell, \"Helvetica Neue\", sans-serif"))), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id => "grid-26e252de",:className => "ag-theme-balham",:style => Dict("height" => "274px","width" => "100%")))], Dict{Symbol,Any}()), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("onCellKeyDown" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onRowClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellDoubleClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellFocused" => (Observable{Any} with 0 listeners. Value:
nothing, nothing)), Set(String[]), nothing, Asset[Asset("js", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/src/rowNumberRenderer.js"), Asset("js", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid.js"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid.css"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid-light.css"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid-dark.css")], Dict{Any,Any}(), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f783f5f8010, Task (runnable) @0x00007f783f5f8010), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = ((function (RowNumberRenderer,agGrid){var gridOptions={\"onCellFocused\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellFocused\",\"scope\":\"14937297495563405910\",\"id\":\"ob_164\",\"type\":\"observable\"},x)}),\"onCellValueChanged\":(function (){return }),\"onCellDoubleClicked\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellDoubleClicked\",\"scope\":\"14937297495563405910\",\"id\":\"ob_162\",\"type\":\"observable\"},x)}),\"multiSortKey\":\"ctrl\",\"columnDefs\":[{\"filter\":false,\"editable\":false,\"headerTooltip\":\"\",\"field\":\"__row__\",\"headerName\":\"Row\",\"sortable\":true,\"type\":\"numericColumn\",\"cellRenderer\":\"rowNumberRenderer\",\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"instance_name\",\"headerName\":\"instance_name\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"Union{Missing, String}\",\"field\":\"FP B\",\"headerName\":\"FP B\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"Union{Missing, String}\",\"field\":\"FP DS\",\"headerName\":\"FP DS\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"Union{Missing, String}\",\"field\":\"FP DS+B\",\"headerName\":\"FP DS+B\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"Union{Missing, String}\",\"field\":\"NP B\",\"headerName\":\"NP B\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"Union{Missing, String}\",\"field\":\"NP DS\",\"headerName\":\"NP DS\",\"sortable\":true,\"type\":null,\"resizable

In [101]:
#pretty_table(wide_ttt; backend = :latex)

In [5]:
# Now let us do the same but without the root node time.
lcttt = let
    sel_columns = [
        :instance_name, :lp_method, :lp_method_switch, :pricing_method, :build_and_solve_time,
        :final_root_time
    ]
    # long (opposed to wide) corrected table of total time
    lcttt = select(lp_method_df, sel_columns)
    lcttt.corrected_time = lcttt.build_and_solve_time .- lcttt.final_root_time
    lcttt.both_timings = tuple.(lcttt.build_and_solve_time, lcttt.corrected_time)
    lcttt.params = join.(zip(lcttt.lp_method, lcttt.lp_method_switch, lcttt.pricing_method), "_")
    pretty_params = Dict{String, String}(
        "barrier_only_none" => "NP B",
        "barrier_switch_furini" => "FP DS+B",
        "simplex_only_furini" => "FP DS",
        "barrier_only_furini" => "FP B",
        "simplex_only_none" => "NP DS"
    )
    lcttt.params = getindex.((pretty_params,), lcttt.params)
    lcttt = select(lcttt, :instance_name, :params, :both_timings)
    lcttt
end
showtable(lcttt)

Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[""], Dict{Symbol,Any}(:style => Dict("height" => "0","padding" => "0","background-color" => "#F5F7F7","color" => "#1c1f20","font-family" => "-apple-system, BlinkMacSystemFont, \"Segoe UI\", Roboto, Oxygen-Sans, Ubuntu, Cantarell, \"Helvetica Neue\", sans-serif"))), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id => "grid-1c188b96",:className => "ag-theme-balham",:style => Dict("height" => "500px","width" => "100%")))], Dict{Symbol,Any}()), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("onCellKeyDown" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onRowClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellDoubleClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellClicked" => (Observable{Any} with 0 listeners. Value:
nothing, nothing),"onCellFocused" => (Observable{Any} with 0 listeners. Value:
nothing, nothing)), Set(String[]), nothing, Asset[Asset("js", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/src/rowNumberRenderer.js"), Asset("js", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid.js"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid.css"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid-light.css"), Asset("css", nothing, "/home/henrique/.julia/packages/TableView/CxWLO/deps/ag-grid-22.1.0/ag-grid-dark.css")], Dict{Any,Any}(), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007fd7e0252980, Task (runnable) @0x00007fd7e0252980), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = ((function (RowNumberRenderer,agGrid){var gridOptions={\"onCellFocused\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellFocused\",\"scope\":\"2746230686756284776\",\"id\":\"ob_09\",\"type\":\"observable\"},x)}),\"onCellValueChanged\":(function (){return }),\"onCellDoubleClicked\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onCellDoubleClicked\",\"scope\":\"2746230686756284776\",\"id\":\"ob_07\",\"type\":\"observable\"},x)}),\"multiSortKey\":\"ctrl\",\"columnDefs\":[{\"filter\":false,\"editable\":false,\"headerTooltip\":\"\",\"field\":\"__row__\",\"headerName\":\"Row\",\"sortable\":true,\"type\":\"numericColumn\",\"cellRenderer\":\"rowNumberRenderer\",\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"instance_name\",\"headerName\":\"instance_name\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"String\",\"field\":\"params\",\"headerName\":\"params\",\"sortable\":true,\"type\":null,\"resizable\":true},{\"filter\":true,\"editable\":false,\"headerTooltip\":\"Tuple{Float64,Float64}\",\"field\":\"both_timings\",\"headerName\":\"both_timings\",\"sortable\":true,\"type\":null,\"resizable\":true}],\"onRowClicked\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.setval({\"name\":\"onRowClicked\",\"scope\":\"2746230686756284776\",\"id\":\"ob_08\",\"type\":\"observable\"},x)}),\"onCellKeyDown\":(function (ev){var x={}; (ev.rowIndex!==undefined) ? (x[\"rowIndex\"]=(ev.rowIndex+1)) : undefined; (ev.colDef!==undefined) ? (x[\"column\"]=ev.colDef.headerName) : undefined; return WebIO.

In [7]:
# Table with both the build+solve time and this time minus the root node solving for each instance (rows)
# and parameter combination (columns).
# wide (opposed to long) corrected table of total time
wcttt = unstack(lcttt, :instance_name, :params, :both_timings)
clean_col(df, col) = setproperty!(df, Symbol(col), (both -> round.(both; digits = 2)).(getproperty(df, Symbol(col))))
clean_col.((wcttt,), ["FP B", "FP DS", "FP DS+B", "NP B", "NP DS"])

pretty_table(wcttt; backend = :latex)
#showtable(wcttt)


\begin{tabular}{rrrrrr}
\hline\hline
\textbf{instance_name} & \textbf{FP B} & \textbf{FP DS} & \textbf{FP DS+B} & \textbf{NP B} & \textbf{NP DS} \\
\texttt{String} & \texttt{Tuple{Float64,Float64}} & \texttt{Tuple{Float64,Float64}} & \texttt{Tuple{Float64,Float64}} & \texttt{Tuple{Float64,Float64}} & \texttt{Tuple{Float64,Float64}} \\\hline
A5 & (NaN, NaN) & (296.42, 261.88) & (260.5, 239.06) & (259.72, 137.4) & (488.51, 54.06) \\
CU1 & (3130.73, 3130.73) & (83.16, 83.16) & (77.97, 77.97) & (26.05, 1.55) & (41.39, 2.49) \\
STS4 & (NaN, NaN) & (106.75, 106.43) & (97.24, 96.71) & (58.82, 14.04) & (131.79, 11.43) \\
STS4s & (NaN, NaN) & (110.62, 110.62) & (101.68, 101.68) & (49.42, 7.2) & (139.08, 6.13) \\
gcut9 & (2958.12, 2956.04) & (21.34, 21.05) & (18.09, 17.35) & (51.63, 9.03) & (253.48, 48.2) \\
okp1 & (NaN, NaN) & (54.78, 53.56) & (53.72, 52.85) & (37.11, 12.8) & (71.77, 9.76) \\
okp4 & (NaN, NaN) & (231.18, 231.17) & (219.45, 219.37) & (82.43, 6.77) & (152.11, 12.36) \\
okp5 & (Na

In [187]:
?setproperty!

search: setproperty!



```
setproperty!(value, name::Symbol, x)
```

The syntax `a.b = c` calls `setproperty!(a, :b, c)`.

See also [`propertynames`](@ref Base.propertynames) and [`getproperty`](@ref Base.getproperty).
